### Imports

In [3]:
import warnings

import numpy as np
import pandas as pd

from sklearn.metrics import log_loss, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import networkx as nx
from tqdm import  tqdm

### Set up WandB

In [4]:
# !pip install wandb -qU

In [5]:
import wandb

In [6]:
wandb.login(key= "d4d576d8b176ec09d0c02bee77f76407e764aa4f")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nici9\_netrc
wandb: Currently logged in as: nicola_c (malthe-pabst-it-universitetet-i-k-benhavn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Read in Motion and Demographics Data

In [ ]:
# MD_df = pd.read_csv("Motion_and_Demographics.csv",index_col = 0)
# MD_df["Run"] = MD_df["scan_id"].str.split("_").str.get(-1)
# MD_df = MD_df.drop("scan_id",axis = 1)
# MD_df = MD_df.drop("Label",axis = 1)
# MD_df["Sex"] = MD_df["Sex"].map({"Male":0,"Female":1})
# MD_df.head(3)

,Sub ID,Age,Sex,IQ,Dataset,Mean_Relative_RMS_Displacement,Max_Relative_RMS_Displacement,Movements_gt_threshold,MeanFD_Power,MeanFD_Jenkinson,MeanDVARS,Run
0,50952,8.740,1.0,134.0,ABIDEI,0.077949,1.330583,23,0.601980,0.359159,2.793079,run-1
1,50953,11.764,1.0,132.0,ABIDEI,0.012573,0.068574,0,0.098726,0.058804,1.987881,run-1
2,50954,14.750,1.0,78.0,ABIDEI,0.008208,0.054994,0,0.053991,0.029231,2.061108,run-1


In [ ]:
# MD_dict = MD_df.set_index(["Sub ID", "Dataset","Run"]).drop_duplicates().to_dict(orient="index")

### Load X,y Data for Train Val Splits

In [ ]:

# #Load dataset
# def load_x_y(dataset:str,binary = "Binary", demographic_features = "With_Demographic", node_features = "All_NF"):
#     #todo 3 types in parameters here
#     if binary == "Multiclass":
#       class_dict = {'TD': 0, 'ASD-ADHD':1, 'ASD':2, 'ADHD':3}
#     elif binary == "Binary":
#       class_dict = {'TD': 0, 'ASD-ADHD':1, 'ASD':1, 'ADHD':1}

#     file_list = pd.read_csv(f'../../data.nosync/networks_multi/networks_multi/{dataset}_set_files.csv')['file'].to_list()
#     data_list = []
#     col_names = ["Sub ID","Run","dataset","y"]

#     for i in tqdm(file_list):
#         i = i.split('/')[2]
#         network_class = class_dict[i.split('_')[3]]
#         id =  i.split('_')[0]
#         run = i.split('_')[1]
#         dataset =  i.split('_')[2]
#         G = nx.read_gml(f'../../data.nosync/networks_multi/networks_multi/{i}')
#         features = [id,run,dataset,network_class]
#         # print("Nodes: ",G.nodes)
#         # print("Class: ",network_class)
#         # print("Features: ",features)
#         # print("Run and dataset: ",run,dataset)
#         # print("MD_info", MD_dict[(int(id),dataset,run)])
#         n_f_names = G.nodes["1"].keys()
#         # print(list(n_f_names))
#         #break
#         # for e in G.edges():
#         #     u, v = e
#         #     for key, value in G[u][v].items():
#         #         G[u][v][key]['edge_features'] = max(value['edge_features'], 0) if not math.isnan(value['edge_features']) else 0
#         # edge_features = nx.get_edge_attributes(G,"edge_features")

#         if node_features == "All_NF":
#           n_f_names = n_f_names
#         elif node_features == "No_PCA_NF":
#           n_f_names = [name for name in n_f_names if  "pca" not in name]
#         elif node_features == "All_PCA_NF":
#           n_f_names = [name for name in n_f_names if  "pca" in name]
#         elif node_features == "One_PCA_NF":
#           n_f_names = [name for name in n_f_names if  "pca_all" in name]
#         elif node_features == "No_NF":
#           n_f_names = []
#         for n in G.nodes():
#           for name in n_f_names:
#             features.append(G.nodes[n][name])

#         if demographic_features == "With_Demographic":
#           for demographic_info in  MD_dict[(int(id),dataset,run)].values():
#             features.append(demographic_info)

#           #features.append(node)
#         data_list.append(features)
#     print(n_f_names)

#     if node_features != "No_NF":
#       for n in G.nodes():
#         col_names.extend([i+"_"+n for i in n_f_names])
#     elif node_features == "No_NF":
#       pass


#     if demographic_features == "With_Demographic":
#       col_names.extend(MD_dict[(int(id),dataset,run)].keys())
#     #[col_names.append(i) for i in range(len(features)-4)]
#     # print(len(col_names),col_names)
#     # print(len(data_list),len(data_list[0]))
#     data_list = pd.DataFrame(data_list,columns=col_names)
#     X =  np.array(data_list[col_names[4:]])
#     y =  np.array(data_list["y"])
#     return X,y

# def load_datasets(binary = "Binary", demographic_features = "With_Demographic", node_features = "All_NF"): # 1 min per call
#   print("Loading: ", binary,demographic_features,node_features)
#   if demographic_features == "Without_Demographic" and node_features == "No_NF":
#     return [],[],[],[]
#   X_train, y_train = load_x_y(dataset='train', binary= binary, demographic_features= demographic_features, node_features=node_features) # 30 sec
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_val, y_val = load_x_y(dataset='val', binary= binary, demographic_features= demographic_features, node_features=node_features)
#   X_val = scaler.transform(X_val)
#   #todo add test?
#   return X_train, y_train, X_val, y_val

In [10]:
# X_train, y_train, X_val, y_val = load_datasets(binary = "Binary", demographic_features = "With_Demographic", node_features = "One_PCA_NF")

#### Create a Dict for fast access to data in the sweep

In [11]:
# # All possibilies for datasets
# Binary = ["Binary","Multiclass"]
# Demographic = ["With_Demographic","Without_Demographic"]
# Node_Features = ["All_NF","No_PCA_NF","All_PCA_NF","One_PCA_NF","No_NF"]

# # a dictionary for fast access to datasets
# dataset_dict = {i:{
#     j:{
#       k:[load_datasets(binary = j, demographic_features = k, node_features = i)] for k in Demographic}
#     for j in Binary }
#                 for i in Node_Features}
# np.save('dataset_dict.npy', dataset_dict) 


In [ ]:
# dataset_dict = np.load('dataset_dict.npy', allow_pickle=True).item()

In [ ]:
# dataset_dict.keys()

dict_keys(['All_NF', 'No_PCA_NF', 'All_PCA_NF', 'One_PCA_NF', 'No_NF'])

In [ ]:
# len(dataset_dict["All_NF"]["Binary"]["With_Demographic"][0])

4

### Train the Models

In [ ]:
rng = np.random.RandomState(0)
def train_random(config=None, binary = False):
  # Train model, get predictions
  with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
    config = wandb.config
    model = DummyClassifier(strategy= config.strategy,
                               random_state=rng)
    
    
    # just giving it any of the datasets (it only uses y anyway)
    X_train, y_train, X_val, y_val = dataset_dict["All_NF"][config.binary]["With_Demographic"][0]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_probas = model.predict_proba(X_val)
    y_probas_train = model.predict_proba(X_train)

      
    n_ll = log_loss(y_val ,y_probas)
    n_ll_train = log_loss(y_train ,y_probas_train)
    f_1 = f1_score(y_pred ,y_val,average = "weighted")

    if binary == False:
      labels = ['TD', 'ASD-ADHD', 'ASD', 'ADHD']
    elif binary == True:
      labels = ["TD","ND"]


    #indices = np.argsort(importances)[::-1]
    print(n_ll,n_ll_train)
    wandb.log({
        "train_loss": n_ll_train,
        "val_loss": n_ll,
        "f1_score": f_1
        })
    wandb.sklearn.plot_classifier(model,
                                X_train, X_val,
                                y_train, y_val,
                                y_pred, y_probas,
                                labels,
                                is_binary=binary,
                                model_name=f'Random_{config.strategy}')



In [ ]:
# run = wandb.init(project='my-scikit-integration', name="logistic_test_balanced_edgefeatures_multi")
# train_random()
# wandb.finish()

In [ ]:
#sweep
Binary = ["Binary","Multiclass"]
Demographic = ["With_Demographic","Without_Demographic"]
Node_Features = ["All_NF","No_PCA_NF","All_PCA_NF","One_PCA_NF","No_NF"]

def main():
    wandb.init(project="logistic_combined_sweep")
    is_binary = wandb.config.binary =="Binary"
    train_random(config=wandb.config, binary = is_binary)
# 2: Define the search space
sweep_configuration = {
    "method": "grid",
    # "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "strategy": {"values": ['most_frequent', 'prior', 'stratified', 'uniform']},
        ## which data demographic and node features
        "binary": {"values":["Binary","Multiclass"]}
    }
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="random_model")



Create sweep with ID: 8kuujfk2
Sweep URL: https://wandb.ai/malthe-pabst-it-universitetet-i-k-benhavn/logistic_testing/sweeps/8kuujfk2


In [18]:
wandb.agent(sweep_id = sweep_id, function=main)

wandb: Agent Starting Run: 5idygwrb with config:
wandb: 	binary: Binary
wandb: 	strategy: most_frequent


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


wandb: 
wandb: Plotting Random_most_frequent.
wandb: Logged feature importances.


14.239468005577146 17.7357659533751


wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

wandb: 
wandb: Plotting Random_uniform.
wandb: Logged feature importances.


1.3862943611198906 1.3862943611198906


wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


f1_score,▁
train_loss,▁
val_loss,▁
f1_score,0.32439
train_loss,1.38629
val_loss,1.38629


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
